In [68]:
# Import modules/packages
import os
import numpy as np

# Import third-party modules/packages
# import utils
from utils import File, Command, NiiFile, LogFile, TmpDir, DependencyError
from typing import Optional, Union, Tuple

In [71]:
class ReconMRtrix (object):
    '''Class that contains the associated wrapper functions for performing
    tractography using MRtrix v3.x.
    
    Attributes:
        *
    '''
    
    nii_file: NiiFile = ""
    bval: File = ""
    bvec: File = ""
    json_file: File = ""
    log: LogFile = ""
    
    def __init__(self,
                 nii_file: str,
                 bval: str,
                 bvec: str,
                 json_file: str = "",
                 log: str = "") -> None:
        '''Init doc string for ReconMRtrix class.
        
        Usage example:
            >>> dwi_obj = ReconMRtrix("file.nii",
            >>>                       "file.bval",
            >>>                       "file.bvec",
            >>>                       "file.json",
            >>>                       "file.log")
            >>> dwi_obj.file
            "file.nii"
        
        Args:
            nii_file: Input file path to DWI NIFTI file.
            bval: Input file path to corresponding bval file.
            bvec: Input file path to corresponding bvec file.
            json_file: Input file path to corresponding JSON (sidecar) file.
            log: Log filename for output log file (, need not exist at runtime).
        '''
        self.nii_file: str = nii_file
        self.bval: str = bval
        self.bvec: str = bvec
        
        self.nii_file = NiiFile(self.nii_file)
        self.bval = File(self.bval)
        self.bvec = File(self.bvec)
        
        if json_file:
            self.json_file: str = json_file
            self.json_file = File(self.json_file)
        else:
            self.json_file = ""
            self.json_file = File(self.json_file)
            
        if log:
            self.log: str = log
            self.log = LogFile(self.log)
        else:
            self.log: str = ""
            self.log = LogFile(self.log)
        
    class Mif(File):
        '''Creates MIF files for use with MRtrix executables. Inherits 
        methods and properties from the File class.
        
        Attributes:
            * 
        '''
        
        def __init__(self,
                     file: File,
                     gzip: bool = False) -> None:
            '''Init doc string for Mif class. Inherits methods and properties
            from the File class. The MIF file can also be gzipped if desired.
            
            Usage example:
                >>> dwi_obj = ReconMRtrix("file.nii",
                >>>                       "file.bval",
                >>>                       "file.bvec",
                >>>                       "file.json",
                >>>                       "file.log")
                >>> dwi_obj.Mif(dwi_obj.file)
                >>> dwi_obj.Mif.file
                >>> "file.mif"
                >>>
                >>> # To gzip the MIF file
                >>> dwi_obj.Mif(dwi_obj.file,gzip=True)
            
            Args:
                file: Input filename of MIF file (need not exist).
                gzip: Gzips output MIF file
            '''
            self.file = file
            
            if '.gz' in self.file:
                if gzip:
                    self.ext: str = self.file[-(7):]
                    self.file = self.file[:-(7)] + ".mif.gz"
                else:
                    self.ext: str = self.file[-(7):]
                    self.file = self.file[:-(7)] + ".mif"
            else:
                if gzip:
                    self.ext: str = self.file[-(4):]
                    self.file = self.file[:-(4)] + ".mif.gz"
                else:
                    self.ext: str = self.file[-(4):]
                    self.file = self.file[:-(4)] + ".mif"
            File.__init__(self,self.file,self.ext)
    
    def nifti_to_mif(self,
                     force: bool = False,
                     gzip: bool = False) -> Mif:
        '''Converts DWI NIFTI file and its associated files to MIF files.
        
        Usage example:
            >>> dwi_obj = ReconMRtrix("file.nii",
            >>>                       "file.bval",
            >>>                       "file.bvec",
            >>>                       "file.json",
            >>>                       "file.log")
            >>> dwi_obj.nifti_to_mif()
            >>> Mif
        
        Args:
            force: Force overwrite of existing MIF file.
            gzip: Gzip output MIF file.
        
        Returns:
            mif_file: Mif file object.
        '''
        mif_file: str = self.nii_file.abs_path()
        mif_file: Mif = self.Mif(mif_file,gzip=gzip)
        
        mr_convert = Command("mrconvert")
        if force:
            mr_convert.cmd_list.append("-force")
        if self.json_file:
            mr_convert.cmd_list.append("-json_import")
            mr_convert.cmd_list.append(self.json_file.file)
        mr_convert.cmd_list.append("-fslgrad")
        mr_convert.cmd_list.append(f"{self.bvec.file}")
        mr_convert.cmd_list.append(f"{self.bval.file}")
        mr_convert.cmd_list.append(f"{self.nii_file.file}")
        mr_convert.cmd_list.append(f"{mif_file.file}")
        
        mr_convert.run(self.log)
        return mif_file
    
    def estimate_response(self,
                          mif: Mif,
                          method: str = "dhollander",
                          force: bool = False
                         ) -> Tuple[str,str,str]:
        '''doc-string
        Todo:
            * function returns should be File objects, not strs
        '''
        [path, filename, ext] = self.nii_file.file_parts()
        
        wm_res = os.path.join(path,filename + "_response_wm.txt")
        gm_res = os.path.join(path,filename + "_response_gm.txt")
        csf_res = os.path.join(path,filename + "_response_csf.txt")
        
        dwi_response = Command("dwi2response")
        if force:
            dwi_response.cmd_list.append("-force")
        dwi_response.cmd_list.append(method)
        dwi_response.cmd_list.append(mif.file)
        dwi_response.cmd_list.append(wm_res)
        dwi_response.cmd_list.append(gm_res)
        dwi_response.cmd_list.append(csf_res)
        
        dwi_response.run(self.log)
        
        return wm_res, gm_res, csf_res
    
    def mr_upsample(self,
                    mif: Mif,
                    vox: float) -> Mif:
        '''doc-string'''
        [path, filename, _ext] = self.nii_file.file_parts()
        [_path, _filename, ext] = mif.file_parts() # Keep original file extension
        
        if ".gz" in mif.file:
            gzip = True
        
        filename = filename + "_upsampled"
        upsampled_mif = os.path.join(path,filename + ext)
        upsampled_mif: Mif = self.Mif(upsampled_mif,gzip=gzip)
        
        try:
            upsample = Command("mrresize")
            upsample.cmd_list.append(mif.file)
            upsample.cmd_list.append("-vox")
            upsample.cmd_list.append(f"{vox}")
            upsample.cmd_list.append(upsampled_mif.file)

            upsample.check_dependency()
            upsample.run(self.log)
            return upsampled_mif
        except DependencyError:
            upsample = Command("mrgrid") 
            upsample.cmd_list.append(mif.file)
            upsample.cmd_list.append("regrid")
            upsample.cmd_list.append(upsampled_mif.file)
            upsample.cmd_list.append("-voxel")
            upsample.cmd_list.append(f"{vox}")
            
            upsample.check_dependency()
            upsample.run(self.log)
            return upsampled_mif
    
    def create_mask(self,
                    mif: Mif,
                    frac_int: float = 0.5) -> Mif:
        '''doc-string'''
        [path, filename, _ext] = self.nii_file.file_parts()
        [_path, _filename, ext] = mif.file_parts() # Keep original file extension
        
        if ".gz" in mif.file:
            gzip = True
        
        filename = filename + "_mask"
        mask_mif = os.path.join(path,filename + ext)
        mask_mif: Mif = self.Mif(mask_mif,gzip=gzip)
            
        # Create temporary directory
        work_dir: TmpDir = TmpDir(tmp_dir=path,use_cwd=False)
        work_dir.mk_tmp_dir()
        
        tmp_b0s: TmpFile = work_dir.TmpFile(tmp_file="tmp_B0s" + _ext,
                                            tmp_dir=work_dir.tmp_dir)
        
        tmp_b0: TmpFile = work_dir.TmpFile(tmp_file="tmp_B0" + _ext,
                                           tmp_dir=work_dir.tmp_dir)
        
        tmp_mask: TmpFile = work_dir.TmpFile(tmp_file=tmp_b0.rm_ext(ext=_ext) + 
                                             _ext,
                                             tmp_dir=work_dir.tmp_dir)
        
        # Extract B0s
        extract_b0s = Command("dwiextract")
        extract_b0s.cmd_list.append("-bzero")
        extract_b0s.cmd_list.append(mif.file)
        extract_b0s.cmd_list.append(tmp_b0s)
        extract_b0s.run(self.log)
        
        # Merge B0s, by obtaining mean of images
        merge_b0s = Command("fslmaths")
        merge_b0s.cmd_list.append(tmp_b0s)
        merge_b0s.cmd_list.append("-Tmean")
        merge_b0s.cmd_list.append(tmp_b0)
        merge_b0s.run(self.log)
        
        # Create brain mask
        bet = Command("bet")
        bet.cmd_list.append(tmp_b0)
        bet.cmd_list.append(tmp_mask)
        bet.cmd_list.append("-R")
        bet.cmd_list.append("-m")
        bet.cmd_list.append("-f")
        bet.cmd_list.append(f"{frac_int}")
        bet.run(self.log)
        
        # Convert NIFTI to MIF
        mr_convert = Command("mrconvert")
        mr_convert.cmd_list.append(tmp_mask)
        mr_convert.cmd_list.append(mask_mif.file)
        mr_convert.run(self.log)
        
        # Clean-up
        work_dir.rm_tmp_dir(rm_parent=True)
        return mask_mif
        
    def ss3t_csd(mif: Mif,
                 mask: Mif,
                 wm_res: File,
                 gm_res: File,
                 csf_res: File
                ) -> Tuple[Mif,Mif,Mif]:
        '''doc-string'''
        [path, filename, _ext] = self.nii_file.file_parts()
        [_path, _filename, ext] = mif.file_parts() # Keep original file extension
        
        if ".gz" in mif.file:
            gzip = True
        
        wm_fod = filename + "_wm_fod"
        gm_tis = filename + "_gm_tis"
        csf_tis = filename + "_csf_tis"
        
        wm_fod: str = os.path.join(path,wm_fod + ext)
        gm_tis: str = os.path.join(path,gm_tis + ext)
        csf_tis: str = os.path.join(path,csf_tis + ext)
        
        wm_fod: Mif = self.Mif(wm_fod,gzip=gzip)
        gm_tis: Mif = self.Mif(gm_tis,gzip=gzip)
        csf_tis: Mif = self.Mif(csf_tis,gzip=gzip)
        
        # Compute WM FOD
        ss3t = Command("ss3t_csd_beta1")
        ss3t.cmd_list.append(mif.file)
        ss3t.cmd_list.append(wm_res.file)
        ss3t.cmd_list.append(wm_fod.file)
        ss3t.cmd_list.append(gm_res.file)
        ss3t.cmd_list.append(gm_tis.file)
        ss3t.cmd_list.append(csf_res.file)
        ss3t.cmd_list.append(csf_tis.file)
        ss3t.cmd_list.append("-mask")
        ss3t.cmd_list.append(mask)
        ss3t.run(self.log)
        return wm_fod, gm_tis, csf_tis
    
    def bias_field_correction(wm_fod,gm,csf,mask):
        '''doc-string'''
    
    def compute_dec_map(wm_fod,mask):
        '''doc-string'''
    
    def mr_trk_global(wm_fod,mask,stream_lines, cutoff):
        '''doc-string'''
    
    # write visualization functions later

In [4]:
class DTKrecon(object):
    '''class doc-string'''
    
    def __init__(self):
        '''doc-string'''
    
    def dti_fit(dwi,mask,bvec,bval,threshold):
        '''doc-string'''
    
    def create_mask(dwi,bval):
        '''doc-string'''
    
    def extract_b0s(dwi,bval):
        '''doc-string'''
        # DWIxfm.extract_b0s
    
    def dti_recon(dwi,mask,threshold):
        '''doc-string'''
    
    # write visualization functions later

In [5]:
class DWIxfm(object):
    '''class doc-string'''
    
    def __init__(self):
        '''doc-string'''
    
    def extract_b0s(dwi,bval):
        '''doc-string'''
    
    def create_mask(dwi,bval):
        '''doc-string'''
    
    def compute_linear_xfm(dwi,template,dof):
        '''doc-string'''
    
    def compute_non_linear_xfm():
        '''doc-string'''
    
    def compute_combined_xfm():
        '''doc-string:
        computes combined linear and non-linear transform.'''
    
    def applywarp():
        '''doc-string'''

In [6]:
def test(file: File):
    pass

In [7]:
f = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz"
b = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bval"
e = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec"
j = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.json"
log = "sub-test.log"

In [60]:
mm = ReconMRtrix(f,b,e,j,log)

In [61]:
mm.nii_file.file

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [62]:
mm.nii_file.abs_path()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [63]:
mf = mm.Mif(mm.nii_file.file)
# mf = mm.Mif(mm.nii_file.file)
mf.ext

'.nii.gz'

In [64]:
# mf.ext = ".mif"; mf.ext

In [65]:
mf.file

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif'

In [55]:
mi = mm.nifti_to_mif(force=True)

Running: mrconvert -force -json_import ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.json -fslgrad ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bval ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif
Running: mrconvert -force -json_import ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.json -fslgrad ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bval ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C0

In [66]:
mi.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif'

In [57]:
[wm_res,gm_res,cxf_res] = mm.estimate_response(mif=mi,force=True)

Running: dwi2response -force dhollander /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_wm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_gm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_csf.txt
Running: dwi2response -force dhollander /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_wm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2

dwi2response: 
dwi2response: Note that this script makes use of commands / algorithms that have relevant articles for citation. Please consult the help page (-help option) for more information.
dwi2response: 
dwi2response: Generated scratch directory: /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/dwi2response-tmp-121NN8/
dwi2response: Importing DWI data (/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif)...
dwi2response: Changing to scratch directory (/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/dwi2response-tmp-121NN8/)
dwi2response: Computing brain mask (dwi2mask)...
dwi2response: -------
dwi2response: 2 unique b-value(s) detected: 0,2000 with 7,61 volumes
dwi2response: -------
dwi2response: Preparation:
dwi2response: * Eroding brain mask by 3 pass(es)...
dwi2response:   [ mask: 119075 -> 80199 ]
dwi2response: * Computing signal decay metric (SDM):
dwi2response:  * b=0...
dw

In [69]:
up_mif = mm.mr_upsample(mif=mi,vox=1.5)

Running: mrgrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif regrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif.gz -voxel 1.5
Running: mrgrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif regrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif.gz -voxel 1.5
Running: mrgrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif regrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_upsampled.mif.gz -voxel 1.5
Running: mrgrid /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti


 	 The required dependency mrresize is not installed or in the system path. 



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

